In [ ]:
!pip install pytorch_lightning
!pip install transformers

In [ ]:
from torch.utils.data import DataLoader, random_split
import os
from pytorch_lightning import Trainer
import torch
from torch.nn import functional as F
from torch import nn
from pytorch_lightning.core.lightning import LightningModule
from torch.optim import Adam
from torch import optim as optim
import random
import math
from torch_sparse import SparseTensor
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from pytorch_lightning.core.lightning import LightningModule
from transformers import AdamW
from torch.nn.parameter import Parameter
from torch_geometric.nn import Sequential, TransformerConv,BatchNorm,Set2Set
from torch.nn.functional import normalize
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.nn.functional as F
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.metrics import f1_score, roc_auc_score, roc_curve,average_precision_score,recall_score,precision_score,matthews_corrcoef
import numpy as np
import pytorch_lightning as pl
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv, GATv2Conv,DenseSAGEConv
from sklearn.model_selection import KFold

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

In [ ]:
#data for interprotein  link prediction
fullsamples=np.load('/workspace/file_name.npy')

xyzl=[]
xyz_outl = []
input_edges_rec = []
input_edges_lig = []
sample_filel = []
maked_inputl = []
len_rec = []
coord_all=[]
edge_index_rec=[]
edge_index_lig=[]
one_hot=[]
feat_in=[]
def sigm(g):
    return(1/(1+math.exp(-g/100)))
for i in range(len(fullsamples)):
    edgesrec = []
    edgeslig = []
#     if fullsamples[i]+'_' in samplesppi4:
    try:
        try:
            datain= torch.load(os.path.join('/workspace/madhav/all_feat/ALL/'+str(fullsamples[i]).split('_')[0].upper()+'.'+str(fullsamples[i]).split('_')[1].split(':')[0]+'.'+str(fullsamples[i]).split('_')[1].split(':')[1]+'.'+'_input_feat.pt'))
        except:
            datain= torch.load(os.path.join('/workspace/madhav/all_feat/ALL/'+str(fullsamples[i]).split('_')[0].lower()+'.'+str(fullsamples[i]).split('_')[1].split(':')[0]+'.'+str(fullsamples[i]).split('_')[1].split(':')[1]+'.'+'_input_feat.pt')) 
        pssm = torch.load(os.path.join('/workspace/pssmdb5ppiprism',fullsamples[i]+'_pssm.pt'))
        edgesrec = torch.load(os.path.join('/workspace/InputEdge_file_nameedge_rec', fullsamples[i]+'_edge_rec.pt'))
        edgeslig = torch.load(os.path.join('/workspace/InputEdge_file_nameedge_lig', fullsamples[i]+'_edge_lig.pt'))
        adjl = torch.load(os.path.join('/workspace/inputfeatdb5ppiprism',fullsamples[i]+'_adj_rec.pt')) 
        adjr = torch.load(os.path.join('/workspace/inputfeatdb5ppiprism',fullsamples[i]+'_adj_lig.pt')) 
        interface = torch.load(os.path.join('/workspace/ReadyToPair', fullsamples[i]+'_interface.pt'))
    except:
        continue
    if not(datain.shape[0]==pssm.shape[0]==interface.shape[0] ) :# or  interface.sum()==0.:
        continue
    feat = torch.hstack((datain, pssm))
    rcc=feat[:, 46].cpu()
    unique, counts = np.unique(rcc, return_counts=True)
    coun = dict(zip(unique, counts))
    rec_c = coun[0.0]
    receptor = feat[:rec_c, 3:]
    ligand = feat[rec_c:, 3:]
    coord=feat[:,:3]
#     one_hot=feat[:,26:46,]
    if receptor.shape[0]>999 or receptor.shape[0]<75 or ligand.shape[0]>999 or ligand.shape[0]<75:
        continue
#     receptor = receptor.expand(receptor.shape[0], receptor.shape[0],receptor.shape[1]).permute(2,1,0)
#     ligand = ligand.expand(ligand.shape[0], ligand.shape[0],ligand.shape[1]).permute(2,1,0)
#     print(i,ligand.shape)
    len_rec.append(rec_c)
    input_edges_rec.append(adjr)
    input_edges_lig.append(adjl)
    edge_index_rec.append(adjr.nonzero().t().contiguous())
    edge_index_lig.append(adjl.nonzero().t().contiguous())
#     print(feat[:,26:46,])
    one_hot.append(feat[:,26:46,])
    feat
    for coor in coord:
        coor[0]=sigm(coor[0])
        coor[1]=sigm(coor[1])
        coor[2]=sigm(coor[2])
    new=torch.cat((feat[:,:26],feat[:,46:]),1)
    feat_in.append(new)
#     print(feat_in.shape)
    new = new[:,3:]
    xyzl.append(new)
    coord_all.append(coord)
    sample_filel.append(i)
     #fullsamples.index(i))
#     input_edges_rec.append(edgesrec)
#     input_edges_lig.append(edgeslig)
    xyz_outl.append(torch.load(os.path.join('/workspace/ReadyToPair', fullsamples[i]+'_interface.pt')))
    print('done')
    #     break
    
len(xyzl), len(xyz_outl), len(len_rec)#, len(input_edges_lig)

In [ ]:
print(feat_in[0].shape)
print(new.shape)

In [ ]:
# print(coord_all[2281])
# torch.set_printoptions(threshold=10_000)
# print(feat.shape)
# print(feat)
# # feat = feat[:,3:]
# print(feat.shape)
# print(feat)
# # print(feat[:,26:46,].shape)
# # print(xyzl[2281])
# print(feat[:,:26],feat[:,46:])
# print('=====================================')
# print(feat[:,:23].shape,feat[:,43:].shape)


In [ ]:
class ProteinDataL(pl.LightningDataModule):
    def __init__(self, node,  xyz_out, len_rec,input_edges_lig,input_edges_rec,coord,edge_index_rec,edge_index_lig,fold,oh): #rec_edge, lig_edge,
        super().__init__()
        self.node = node
        self.xyz_out = xyz_out
        self.rec_edge = input_edges_rec
        self.lig_edge = input_edges_lig
        self.len_rec = len_rec
        self.coord   =coord
        self.edge_index_rec=edge_index_rec
        self.edge_index_lig=edge_index_lig
        self.fold=fold
        self.one_hot=oh
        self.setup()
        
    def setup(self, stage = None):
        counto=0
        together=[]
        self.data= []
        train_set=[]
        test_set=[]
        for i in range(len(self.node)):
            self.data.append([
                              self.node[i].float().clone().detach(),
                              self.xyz_out[i].float().clone().detach(),
                              self.coord[i].float().clone().detach(),
                              self.lig_edge[i].float().clone().detach(),
                              self.rec_edge[i].float().clone().detach(),
                              self.edge_index_lig[i].float().clone().detach(),
                              self.edge_index_rec[i].float().clone().detach(),
                              self.len_rec[i],
                              self.one_hot[i]])
        
        kf = KFold(n_splits=5, shuffle=True, random_state=11001)    
        for tog in kf.split(self.data):
            together=together+[tog]
        ti,vi=together[self.fold]
        self.tii, self.vii = ti.tolist(), vi.tolist()
        for i in self.vii:
            test_set.append(self.data[i])
        for i in self.tii:
            train_set.append(self.data[i])


        self.train_set,self.val_set=train_set,test_set

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=1,num_workers=32)

#     def val_dataloader(self):
#         return DataLoader(self.val_set, batch_size=1,num_workers=32)

    def test_dataloader(self):
        return DataLoader(self.val_set, batch_size=1,num_workers=32)



In [ ]:
from torch.nn import Parameter


from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import zeros

class DenseGCNConv(torch.nn.Module):
    r"""See :class:`torch_geometric.nn.conv.GCNConv`.
    """
    def __init__(self, in_channels, out_channels, improved=False, bias=True):
        super().__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved

        self.lin = Linear(in_channels, out_channels, bias=False,
                          weight_initializer='glorot')

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        stdv = 1. / math.sqrt(self.out_channels)
        self.bias.data.uniform_(-stdv, stdv)


    def forward(self, x, adj, mask=None, add_loop=True):
        r"""
        Args:
            x (Tensor): Node feature tensor :math:`\mathbf{X} \in \mathbb{R}^{B
                \times N \times F}`, with batch-size :math:`B`, (maximum)
                number of nodes :math:`N` for each graph, and feature
                dimension :math:`F`.
            adj (Tensor): Adjacency tensor :math:`\mathbf{A} \in \mathbb{R}^{B
                \times N \times N}`. The adjacency tensor is broadcastable in
                the batch dimension, resulting in a shared adjacency matrix for
                the complete batch.
            mask (BoolTensor, optional): Mask matrix
                :math:`\mathbf{M} \in {\{ 0, 1 \}}^{B \times N}` indicating
                the valid nodes for each graph. (default: :obj:`None`)
            add_loop (bool, optional): If set to :obj:`False`, the layer will
                not automatically add self-loops to the adjacency matrices.
                (default: :obj:`True`)
        """
        x = x.unsqueeze(0) if x.dim() == 2 else x
        adj = adj.unsqueeze(0) if adj.dim() == 2 else adj
        B, N, _ = adj.size()

        if add_loop:
            adj = adj.clone()
            idx = torch.arange(N, dtype=torch.long, device=adj.device)
            adj[:, idx, idx] = 1 if not self.improved else 2

        out = self.lin(x)
        deg_inv_sqrt = adj.sum(dim=-1).clamp(min=1).pow(-0.5)

        adj = deg_inv_sqrt.unsqueeze(-1) * adj * deg_inv_sqrt.unsqueeze(-2)
        out = torch.matmul(adj, out)

        if self.bias is not None:
            out = out + self.bias

        if mask is not None:
            out = out * mask.view(B, N, 1).to(x.dtype)

        return out



In [ ]:
# from torch_geometric.nn import DenseGCNConv,DenseGINConv,DenseGraphConv,DenseSAGEConv
class NetR(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = DenseGCNConv(128, 128) 
        self.l2 = DenseGCNConv(256, 256)
        self.l3 = DenseGCNConv(128, 128)
        self.l4 = DenseGCNConv(64, 64)
        self.l5 = DenseGCNConv(1024, 1024)
        self.l6 = DenseGCNConv(512, 512)
        self.l7 = DenseGCNConv(256, 256)
        self.l8 = DenseGCNConv(128, 128)
        self.l9 = DenseGCNConv(64, 64)
        self.attn1 = nn.MultiheadAttention(128, 4)
        self.attn2 = nn.MultiheadAttention(256, 4)
        self.attn3 = nn.MultiheadAttention(128, 4)
        self.relu = nn.LeakyReLU()
        self.sig =nn.Sigmoid()
        embed_dim = 512
        num_heads = 4
        self.o0=nn.LayerNorm(128)
        self.o1=nn.LayerNorm(256)
        self.o2=nn.LayerNorm(128)
        self.o3=nn.LayerNorm(64)
        self.b0=nn.Bilinear(164,128,256)
        self.b1=nn.Bilinear(256,128,64)

        self.h0 = nn.Linear(164,128)
#         self.h1 = nn.Linear(128,256)
        self.h2 = nn.Linear(256,128)
#         self.h3 = nn.Linear(128,64)

#         self.h4 = nn.Linear(2048,1024)
#         self.h5 = nn.Linear(1024,512)
#         self.h6 = nn.Linear(512,256)
#         self.h7 = nn.Linear(256,128)
#         self.h8 = nn.Linear(128,64)
        self.h9 = nn.Linear(64,64)
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.dropout = 0.25
    def forward(self, x, adjacency):
        x = F.dropout(x, self.dropout, training=self.training)

        x1 =self.relu(self.h0(x))#64->100
        x1 = self.o0(x1)
        x1 = self.relu(self.l1( x1, adjacency)+x1) 
        x1 = F.dropout(x1, self.dropout, training=self.training)
        x1,_= self.attn1(x1, x1 ,x1)
        x1=x1.squeeze(0)
        
        x2 =self.relu(self.b0(x,x1))##100->380,200
        x2 = self.o1(x2)
        x2 = self.relu(self.l2( x2, adjacency)+x2)
        x2= F.dropout(x2, self.dropout, training=self.training)
        x2,_= self.attn2(x2, x2 ,x2)
        x2=x2.squeeze(0)
        
        x3 =self.relu(self.h2(x2))
        x3 = self.o2(x3)
        x3 = self.relu(self.l3( x3, adjacency)+x3)
        x3 = F.dropout(x3, self.dropout, training=self.training)
        x3,_= self.attn3(x3, x3 ,x3)
        x3=x3.squeeze(0)
        
        x4 =self.relu(self.b1(x2,x3))
        x4 = self.o3(x4)
        x4 = self.relu(self.l4( x4, adjacency)+x4)
        x4 = F.dropout(x4, self.dropout, training=self.training)
#         h4 =self.relu(self.h4(x4))

#         x5 = self.relu(self.l5( h4, adjacency))
#         x5,_= self.attn3(x5, x5 ,x5)

#         x5 = F.layer_norm(x5,[x5.size()[-1]])
#         x5 = F.dropout(x5, self.dropout, training=self.training)
#         h5 =self.relu(self.h5(x5))

#         x6 = self.relu(self.l6( h5, adjacency))
#         x6,_= self.attn2(x6, x6 ,x6)
#         x6 = F.layer_norm(x6,[x6.size()[-1]])
#         x6 = F.dropout(x6, self.dropout, training=self.training)
#         h6 =self.relu(self.h6(x6))
#         x7 = self.relu(self.l7( h6, adjacency))
#         x7,_= self.attn1(x7, x7 ,x7)
#         x7 = F.layer_norm(x7,[x7.size()[-1]])
#         x7 = F.dropout(x7, self.dropout, training=self.training)
#         h7 =self.relu(self.h7(x7))
#         x8 = self.relu(self.l8( h7, adjacency))
#         x8 = F.layer_norm(x8,[x8.size()[-1]])
#         x8 = F.dropout(x8, self.dropout, training=self.training)
#         h8 =self.relu(self.h8(x8))
#         x9 = self.relu(self.l9(h8, adjacency))
#         x9 = F.layer_norm(x9,[x9.size()[-1]])
#         x9 = F.dropout(x9, self.dropout, training=self.training)
                

        return x4


In [ ]:
# from torch_geometric.nn import DenseGCNConv,DenseGINConv,DenseGraphConv,DenseSAGEConv
class NetL(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = DenseGCNConv(128, 128) 
        self.l2 = DenseGCNConv(256, 256)
        self.l3 = DenseGCNConv(128, 128)
        self.l4 = DenseGCNConv(64, 64)
        self.l5 = DenseGCNConv(1024, 1024)
        self.l6 = DenseGCNConv(512, 512)
        self.l7 = DenseGCNConv(256, 256)
        self.l8 = DenseGCNConv(128, 128)
        self.l9 = DenseGCNConv(64, 64)
        self.attn1 = nn.MultiheadAttention(128, 4)
        self.attn2 = nn.MultiheadAttention(256, 4)
        self.attn3 = nn.MultiheadAttention(128, 4)
        self.relu = nn.LeakyReLU()
        self.sig =nn.Sigmoid()
        embed_dim = 512
        num_heads = 4
        self.o0=nn.LayerNorm(128)
        self.o1=nn.LayerNorm(256)
        self.o2=nn.LayerNorm(128)
        self.o3=nn.LayerNorm(64)
        self.b0=nn.Bilinear(164,128,256)
        self.b1=nn.Bilinear(256,128,64)

        self.h0 = nn.Linear(164,128)
#         self.h1 = nn.Linear(128,256)
        self.h2 = nn.Linear(256,128)
#         self.h3 = nn.Linear(128,64)

#         self.h4 = nn.Linear(2048,1024)
#         self.h5 = nn.Linear(1024,512)
#         self.h6 = nn.Linear(512,256)
#         self.h7 = nn.Linear(256,128)
#         self.h8 = nn.Linear(128,64)
        self.h9 = nn.Linear(64,64)
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.dropout = 0.25
    def forward(self, x, adjacency):
        x = F.dropout(x, self.dropout, training=self.training)

        x1 =self.relu(self.h0(x))#64->100
        x1 = self.o0(x1)
        x1 = self.relu(self.l1( x1, adjacency)+x1) 
        x1 = F.dropout(x1, self.dropout, training=self.training)
        x1,_= self.attn1(x1, x1 ,x1)
        x1=x1.squeeze(0)
        
        x2 =self.relu(self.b0(x,x1))##100->380,200
        x2 = self.o1(x2)
        x2 = self.relu(self.l2( x2, adjacency)+x2)
        x2= F.dropout(x2, self.dropout, training=self.training)
        x2,_= self.attn2(x2, x2 ,x2)
        x2=x2.squeeze(0)
        
        x3 =self.relu(self.h2(x2))
        x3 = self.o2(x3)
        x3 = self.relu(self.l3( x3, adjacency)+x3)
        x3 = F.dropout(x3, self.dropout, training=self.training)
        x3,_= self.attn3(x3, x3 ,x3)
        x3=x3.squeeze(0)
        
        x4 =self.relu(self.b1(x2,x3))
        x4 = self.o3(x4)
        x4 = self.relu(self.l4( x4, adjacency)+x4)
        x4 = F.dropout(x4, self.dropout, training=self.training)
#         h4 =self.relu(self.h4(x4))

#         x5 = self.relu(self.l5( h4, adjacency))
#         x5,_= self.attn3(x5, x5 ,x5)

#         x5 = F.layer_norm(x5,[x5.size()[-1]])
#         x5 = F.dropout(x5, self.dropout, training=self.training)
#         h5 =self.relu(self.h5(x5))

#         x6 = self.relu(self.l6( h5, adjacency))
#         x6,_= self.attn2(x6, x6 ,x6)
#         x6 = F.layer_norm(x6,[x6.size()[-1]])
#         x6 = F.dropout(x6, self.dropout, training=self.training)
#         h6 =self.relu(self.h6(x6))
#         x7 = self.relu(self.l7( h6, adjacency))
#         x7,_= self.attn1(x7, x7 ,x7)
#         x7 = F.layer_norm(x7,[x7.size()[-1]])
#         x7 = F.dropout(x7, self.dropout, training=self.training)
#         h7 =self.relu(self.h7(x7))
#         x8 = self.relu(self.l8( h7, adjacency))
#         x8 = F.layer_norm(x8,[x8.size()[-1]])
#         x8 = F.dropout(x8, self.dropout, training=self.training)
#         h8 =self.relu(self.h8(x8))
#         x9 = self.relu(self.l9(h8, adjacency))
#         x9 = F.layer_norm(x9,[x9.size()[-1]])
#         x9 = F.dropout(x9, self.dropout, training=self.training)
                

        return x4


In [ ]:
class NetCoord(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = DenseGCNConv(32, 32) 
        self.l2 = DenseGCNConv(64, 64)
        self.l3 = DenseGCNConv(128, 128)
        self.l4 = DenseGCNConv(64, 64)
        self.l5 = DenseGCNConv(32, 32)
        self.l6 = DenseGCNConv(64, 64)
        self.l7 = DenseGCNConv(32, 32)
        self.l8 = DenseGCNConv(128, 128)
        self.l9 = DenseGCNConv(64, 64)
        self.attn1 = nn.MultiheadAttention(32, 8)
        self.attn2 = nn.MultiheadAttention(64, 8)
        self.attn3 = nn.MultiheadAttention(128, 8)
        self.relu = nn.LeakyReLU()
        embed_dim = 512
        num_heads = 4
        self.h0 = nn.Linear(3,32)
        self.h1 = nn.Linear(32,64)
        self.h2 = nn.Linear(64,128)
        self.h3 = nn.Linear(128,64)
        self.h4 = nn.Linear(64,32)
        self.h5 = nn.Linear(32,3)

        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.dropout = (0.3)
    def forward(self, x, adjacency):
        x = F.dropout(x, self.dropout, training=self.training)
        h0 =self.relu(self.h0(x))#64->100
        x1 = self.relu(self.l1( h0, adjacency)) 
        x1 = F.layer_norm(x1,[x1.size()[-1]])
        x1 = F.dropout(x1, self.dropout, training=self.training)
        h1 =self.relu(self.h1(x1))##100->380,200
        x2 = self.relu(self.l2( h1, adjacency))
        x2 = F.layer_norm(x2,[x2.size()[-1]])
        x2= F.dropout(x2, self.dropout, training=self.training)
        h2 =self.relu(self.h2(x2))
        x3 = self.relu(self.l3( h2, adjacency))
        x3 = F.layer_norm(x3,[x3.size()[-1]])
        x3 = F.dropout(x3, self.dropout, training=self.training)
        h3 =self.relu(self.h3(x3))
        x4 = self.relu(self.l4( h3, adjacency))
        x4 = F.layer_norm(x4,[x4.size()[-1]])
        x4 = F.dropout(x4, self.dropout, training=self.training)
        h4 =self.relu(self.h4(x4))

        x5 = self.relu(self.l5( h4, adjacency))

        x5 = F.layer_norm(x5,[x5.size()[-1]])
        x5 = F.dropout(x5, self.dropout, training=self.training)
#         x5 =self.relu(self.h5(x5))



        return x5

In [ ]:
class GeneratorL(LightningModule):
    def __init__(self):
            super(GeneratorL, self).__init__()
            self.Amean = []
            self.Lmean = []
            self.wholeAmean = []
            self.wholeLmean = []
            self.model= NetL() #Attention_UNET(44,1)
            
          #why are we using conv2d?                      

    def forward(self, x, edge_rec):
        return self.model( x, edge_rec)


In [ ]:
class GeneratorR(LightningModule):
    def __init__(self):
            super(GeneratorR, self).__init__()
            self.Amean = []
            self.Lmean = []
            self.wholeAmean = []
            self.wholeLmean = []
            self.model= NetR() #Attention_UNET(44,1)

          #why are we using conv2d?                      

    def forward(self, x, edge_rec):
        return self.model( x, edge_rec)


In [ ]:
class GeneratorCoord(LightningModule):
    def __init__(self):
            super(GeneratorCoord, self).__init__()
            self.Amean = []
            self.Lmean = []
            self.wholeAmean = []
            self.wholeLmean = []
            self.model= NetCoord() #Attention_UNET(44,1)

          #why are we using conv2d?                      

    def forward(self, x, edge_rec):
        return self.model( x, edge_rec)

In [ ]:

class LinkPred(LightningModule):
    def __init__(self):
        super(LinkPred, self).__init__()
        self.Amean = []
        self.Lmean = []
        self.wholeAmean = []
        self.wholeLmean = []
    
        output_ch = 64
        self.multihead_attn = nn.MultiheadAttention( output_ch, 4)

        self.h_feats_dim = 96
        self.att_mlp_Q = nn.Sequential(nn.Linear(self.h_feats_dim, self.h_feats_dim, bias=True),\
                                  nn.LeakyReLU())# get_non_lin(nonlin, leakyrelu_neg_slope),
        self.att_mlp_K = nn.Sequential(nn.Linear(self.h_feats_dim, self.h_feats_dim, bias=True),\
                                  nn.LeakyReLU())#get_non_lin(nonlin, leakyrelu_neg_slope),
        self.att_mlp_V = nn.Linear(self.h_feats_dim, self.h_feats_dim, bias=True)
        self.cross_msgs = True
        self.coordlayer=GeneratorCoord()
        self.modelL=GeneratorL()
        self.modelR=GeneratorR()
        self.ls=nn.LSTM(20,60,num_layers=3,bias=True,bidirectional=True)
        self.layer_2 = nn.Sequential(nn.Linear(self.h_feats_dim, 128), nn.LeakyReLU(), nn.Linear(128, 256),\
                    nn.LeakyReLU(), nn.Linear(256, 1))
    

    def forward(self, x,edge,no):
        if no==1:
            return self.modelR(x,edge)
        else:
            return self.modelL(x,edge)          

    def training_step(self, batch, batch_idx):
        x,  y,coord,r_edge,l_edge,edge_i_l,edge_i_r,rec_len,oh = batch ##x:1,L,44 y:1,L

        l_edge=l_edge.squeeze(0)
        r_edge=r_edge.squeeze(0)
        receptor = x[0][:rec_len,:]
        ligand  = x[0][rec_len:,:]
        coord_rec=(coord.squeeze(0)[:rec_len,:])
        coord_lig=(coord.squeeze(0)[rec_len:,:])
        edge_i_r=edge_i_r.squeeze(0)
        edge_i_l=edge_i_l.squeeze(0)
        oh=oh.squeeze(0)
        
        output_rec,(hs_r,cs_r)=self.ls(oh[:rec_len,:].unsqueeze(0).float())
        output_lig,(hs_l,cs_l)=self.ls(oh[rec_len:,:].unsqueeze(0).float())
        output_rec=output_rec.squeeze(0) 
        output_lig=output_lig.squeeze(0)
    
        
        outr_coord=self.coordlayer(coord_rec,r_edge)
        receptor=torch.cat((receptor,output_rec),1)
        
        outr = self(receptor.float(),r_edge,1)#.permute(1,2,0)#[0] #L,L,64

        outr, _ = self.multihead_attn(outr, outr, outr) #.to(device)
        outr=torch.cat((outr_coord,outr),2)
        
        ligand=torch.cat((ligand,output_lig),1)
        outl = self(ligand.float(),l_edge,0)#.permute(1,2,0)#[0] #L,L,64
        outl_coord=self.coordlayer(coord_lig,l_edge)
        
        outl, _ = self.multihead_attn(outl, outl, outl) #.to(device)
        outl=torch.cat((outl_coord,outl),2)  

        assert torch.isfinite(outr).any()
        assert torch.isfinite(outl).any()
        
        
        rec, lig = self.cross_attention_layer(outr[0].to(device), outl[0].to(device))

        com = torch.cat((rec, lig),0)
#         com = com.transpose(1,0).unsqueeze(0).unsqueeze(-1)
#         print('concat')
        com = self.layer_2(com)
        com=com.squeeze(-1)
        out=com
        
#         print('layer2 over', com.get_device())
#         out = torch.transpose(com, 1,0)
#         out = com[0][0].transpose(1,0)
        loss = 1.0*F.binary_cross_entropy_with_logits(out.float(), y[0].float(),\
                pos_weight=torch.FloatTensor([(y.size()[1]-float(y.sum()))*1.0/float(y.sum())]).type_as(out.float()))   
        val = 0. # this is the value where you want the data to appear on the y-axis.
        loss=loss.mean()


        binoutput=torch.where(out>=0.5, 1., 0.)
#         print('binoutput', binoutput.get_device())
        
#         print(binoutput.shape,receptor.shape, ligand.shape, 'binshape' )
        one_index_out = torch.nonzero(binoutput).flatten().tolist()
#         print('one_index_out')
        one_index_y = torch.nonzero(y[0]).flatten().tolist()
#         print('one_index_y')
        accuaracy = len(list(set(one_index_out) & set(one_index_y)))/len(one_index_y)
#         print('accuarcyok')
        result_dict =  {'loss': loss, 'accuracy': accuaracy, 'in':out.unsqueeze(0), 'out':y}
        
# # #         loss=self.criterion(out[0].float(),y[0].float()) #+self.compute_body_intersection_loss(transformed, rec_coord)
        return result_dict #loss
    def train_epoch_end(self, outputs):
            accuracy = []
            loss = []

            for i in outputs:
                accuracy.append(i['accuracy'])
                loss.append(i['loss'])
 
            avg_accuracy= torch.mean(torch.tensor(accuracy).float())
            avg_loss = torch.mean(torch.tensor(loss))

            print('accuracy ',avg_accuracy,'loss',avg_loss)


    def test_step(self, batch, batch_idx):
        x,  y,coord,r_edge,l_edge,edge_i_l,edge_i_r,rec_len,oh = batch ##x:1,L,44 y:1,L

        l_edge=l_edge.squeeze(0)
        r_edge=r_edge.squeeze(0)
        receptor = x[0][:rec_len,:]
        ligand  = x[0][rec_len:,:]
        coord_rec=(coord.squeeze(0)[:rec_len,:])
        coord_lig=(coord.squeeze(0)[rec_len:,:])
        edge_i_r=edge_i_r.squeeze(0)
        edge_i_l=edge_i_l.squeeze(0)
        oh=oh.squeeze(0)
        
        output_rec,(hs_r,cs_r)=self.ls(oh[:rec_len,:].unsqueeze(0).float())
        output_lig,(hs_l,cs_l)=self.ls(oh[rec_len:,:].unsqueeze(0).float())
        output_rec=output_rec.squeeze(0) 
        output_lig=output_lig.squeeze(0)
    
        
        outr_coord=self.coordlayer(coord_rec,r_edge)
        receptor=torch.cat((receptor,output_rec),1)
        
        outr = self(receptor.float(),r_edge,1)#.permute(1,2,0)#[0] #L,L,64

        outr, _ = self.multihead_attn(outr, outr, outr) #.to(device)
        outr=torch.cat((outr_coord,outr),2)
        
        ligand=torch.cat((ligand,output_lig),1)
        outl = self(ligand.float(),l_edge,0)#.permute(1,2,0)#[0] #L,L,64
        outl_coord=self.coordlayer(coord_lig,l_edge)
        
        outl, _ = self.multihead_attn(outl, outl, outl) #.to(device)
        outl=torch.cat((outl_coord,outl),2)  

        assert torch.isfinite(outr).any()
        assert torch.isfinite(outl).any()
        
        
        rec, lig = self.cross_attention_layer(outr[0].to(device), outl[0].to(device))

        com = torch.cat((rec, lig),0)
        com = self.layer_2(com)
        com=com.squeeze(-1)
        out=com
        

        loss = 1.0*F.binary_cross_entropy_with_logits(out.float(), y[0].float(),\
        pos_weight=torch.FloatTensor([(y.size()[1]-float(y.sum()))*1.0/float(y.sum())]).type_as(out.float()))    #.unsqueeze(0)
        loss= loss.mean()
        binoutput=torch.where(out>=0.5, 1., 0.)

        cpuout = binoutput.flatten().cpu().detach().numpy()
        cpuy = y.flatten().cpu().detach().numpy()
        auc= roc_auc_score(cpuy, cpuout)
#         accuracy = accuracy_score(cpuy, cpuout)
        F1_score = f1_score(cpuy, cpuout)
        recall= recall_score(cpuy, cpuout) #binoutput) #
        precision= precision_score(cpuy, cpuout)
        mcc= matthews_corrcoef(cpuy, cpuout)
        aucpr=average_precision_score(cpuy,cpuout)

        return { 'mcc': mcc,'Precision':precision, 'Recall':recall ,'F1_score':F1_score,'auroc':auc,'auc pr':aucpr} #'loss':loss,
        
    def test_epoch_end(self, outputs):
        accuracy = []
        loss = []
        Mcc = []
        Precision = []
        Recall =[]
        F1_score = []
        auroc = []
        aucpr=[]
        for i in outputs:
            Mcc.append(i['mcc'])
            Precision.append(i['Precision'])
            Recall.append(i['Recall'])
            F1_score.append(i['F1_score'])
            auroc.append(i['auroc'])
            aucpr.append(i['auc pr'])
        avg_accuracy= torch.mean(torch.tensor(accuracy).float())
        avg_mcc = torch.mean(torch.tensor(Mcc).float())
        avg_Precision = torch.mean(torch.tensor(Precision).float())
        avg_Recall = torch.mean(torch.tensor(Recall).float())
        avg_F1_score = torch.mean(torch.tensor(F1_score).float())
        avg_auroc = torch.mean(torch.tensor(auroc).float())
        avg_aucpr = torch.mean(torch.tensor(aucpr).float())

        print( 'MCC: ',avg_mcc, 'P: ',avg_Precision,'R: ',avg_Recall,\
             'F1: ', avg_F1_score, 'AUROC: ',avg_auroc, 'AUCPR:',avg_aucpr)







    def compute_cross_attention(self, queries, keys, values, mask, cross_msgs):

        if not self.cross_msgs:
            return queries * 0.
        mask = mask.to(device)
        a = mask * torch.mm(queries, torch.transpose(keys, 1, 0)) - 1000. * (1. - mask)
        a_x = torch.softmax(a, dim=1)  # i->j, NxM, a_x.sum(dim=1) = torch.ones(N)
        attention_x = torch.mm(a_x, values)  # (N,d)
        return attention_x



    def get_mask(self, ligand_batch_num_nodes, receptor_batch_num_nodes):
        rows = ligand_batch_num_nodes# sum()
        cols = receptor_batch_num_nodes #sum()
        mask = torch.ones(rows, cols)#.to(device)
        return mask
    def cross_attention_layer(self, rec_feat, lig_feat):
#         before = torch.cat((rec_feat, lig_feat), 0)
        
        mask = self.get_mask( lig_feat.shape[0], rec_feat.shape[0])
        cross_ligand = self.compute_cross_attention(self.att_mlp_Q(lig_feat),
                                self.att_mlp_K(rec_feat),
                                self.att_mlp_V(rec_feat),
                                mask,
                                self.cross_msgs)
        cross_receptor = self.compute_cross_attention(self.att_mlp_Q(rec_feat),
                                self.att_mlp_K(lig_feat),
                                self.att_mlp_V(lig_feat),
                                mask.transpose(0,1),
                                self.cross_msgs)
        cross_receptor += rec_feat
        cross_ligand += lig_feat
        
        
        return cross_receptor, cross_ligand  #combine

    def training_epoch_end(self, outputs):
        self.Amean = []
        self.Lmean = []
        
        for i in outputs:
            self.Amean.append(i['accuracy'])
            self.Lmean.append(i['loss'])
        self.wholeAmean.append(torch.mean(torch.tensor(self.Amean)))
        self.wholeLmean.append(torch.mean(torch.tensor(self.Lmean)))
        print(self.wholeLmean[-1])
        self.logger.experiment.add_scalar('loss',torch.mean(torch.tensor(self.Amean)))
    
    def configure_optimizers(self):
#         optimizer = optim.Adam(self.parameters(), lr=0.0005 , betas=(0.1, 0.999)) #self.lr
        optimizer =  optim.RAdam(self.parameters(), lr=0.00005)
        return optimizer

modelpred = LinkPred()#.to(device) avg_pos_wt


# logger = TensorBoardLogger("tb_logs", name="model")
# trainer = Trainer(max_epochs=250,gpus=[2])
# trainer.fit(modelpred,dataobjl)
# trainer.save_checkpoint("linkpred_250_multi.ckpt")
# trainer.test(modelpred, dataobjl) 
# modelpred    

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,0,one_hot)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=3)
trainer.fit(modelpred,dataobjl)

In [ ]:
trainer.test(modelpred,dataobjl)


In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,0)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=100)
trainer.fit(modelpred,dataobjl)
trainer.test(modelpred,dataobjl)
trainer.save_checkpoint("kfoldboii1.ckpt")

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,1)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=100)
trainer.fit(modelpred,dataobjl)
trainer.test(modelpred,dataobjl)
trainer.save_checkpoint("kfoldboii2.ckpt")

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,2)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=100)
trainer.fit(modelpred,dataobjl)
trainer.test(modelpred,dataobjl)
trainer.save_checkpoint("kfoldboii3.ckpt")

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,3)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=100)
trainer.fit(modelpred,dataobjl)
trainer.test(modelpred,dataobjl)
trainer.save_checkpoint("kfoldboii4.ckpt")

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,4)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=100)
trainer.fit(modelpred,dataobjl)
trainer.test(modelpred,dataobjl)
trainer.save_checkpoint("kfoldboii5.ckpt")

In [ ]:
#epipred test with no rotation

In [ ]:
fullsamples=np.load('/workspace/testEpi.npy')

xyzl=[]
xyz_outl = []
input_edges_rec = []
input_edges_lig = []
sample_filel = []
maked_inputl = []
len_rec = []
coord_all=[]
edge_index_rec=[]
edge_index_lig=[]
sample_file=[]
def sigm(g):
    return(1/(1+math.exp(-g/100)))
for i in range(len(fullsamples)):
    
    edgesrec = []
    edgeslig = []
#     if fullsamples[i]+'_' in samplesppi4:
    try:
        datain = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_input_feat.pt'))
        pssm = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_pssm.pt'))
        edgesrec = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_edge_rec.pt'))
        edgeslig = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_edge_lig.pt'))
        adjl = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_adj_rec.pt')) 
        adjr = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_adj_lig.pt')) 
        interface = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_interface.pt'))
    except:
        continue
    if not(datain.shape[0]==pssm.shape[0]==interface.shape[0] ) :# or  interface.sum()==0.:
        continue
    feat = torch.hstack((datain, pssm))
    rcc=feat[:, 46].cpu()
    unique, counts = np.unique(rcc, return_counts=True)
    coun = dict(zip(unique, counts))
    rec_c = coun[0.0]
    receptor = feat[:rec_c, 3:]
    ligand = feat[rec_c:, 3:]
    coord=feat[:,:3]
    if receptor.shape[0]>999 or receptor.shape[0]<75 or ligand.shape[0]>999 or ligand.shape[0]<75:
        continue
#     receptor = receptor.expand(receptor.shape[0], receptor.shape[0],receptor.shape[1]).permute(2,1,0)
#     ligand = ligand.expand(ligand.shape[0], ligand.shape[0],ligand.shape[1]).permute(2,1,0)
#     print(i,ligand.shape)
    len_rec.append(rec_c)
    input_edges_rec.append(adjr)
    input_edges_lig.append(adjl)
    edge_index_rec.append(adjr.nonzero().t().contiguous())
    edge_index_lig.append(adjl.nonzero().t().contiguous())
    for coor in coord:
        coor[0]=sigm(coor[0])
        coor[1]=sigm(coor[1])
        coor[2]=sigm(coor[2])

#     x1,x2,x3 = torch.from_numpy(np.random.uniform(3.14/12, 3.14, 3))
#     cos1=torch.cos(x1)
#     cos2=torch.cos(x2)
#     cos3=torch.cos(x3)
#     sin1=torch.sin(x1)
#     sin2=torch.sin(x2)
#     sin3=torch.sin(x3)
#     RMatrix=torch.tensor([[cos1*cos2*cos3-sin1*sin3,cos3*cos2*sin1+sin3+cos1,-cos3*sin2 ],\
#                             [-sin3*cos2*cos1-cos3*sin1, -sin3*cos2*sin1+cos3*cos1, sin3*sin2],\
#                             [sin2*cos1,sin2*sin1,cos2]])

#     ligcoord=  ((RMatrix @ torch.tensor(coord[rec_c:,]).double().T)).T
#     reccoord=coord[:rec_c,:]

#     coord=torch.cat((ligcoord,reccoord),0)
    feat = feat[:,3:]
    xyzl.append(feat)
    coord_all.append(coord)
    sample_filel.append(i)#fullsamples.index(i))
#     input_edges_rec.append(edgesrec)
#     input_edges_lig.append(edgeslig)
    xyz_outl.append(torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_interface.pt')))
    sample_file.append((fullsamples[i]+'1',fullsamples[i]+'2'))

    #     break
    
len(xyzl), len(xyz_outl), len(len_rec)#, len(input_edges_lig)

In [ ]:
class ProteinDataL(pl.LightningDataModule):
    def __init__(self, node,  xyz_out, len_rec,input_edges_lig,input_edges_rec,coord,edge_index_rec,edge_index_lig,fold): #rec_edge, lig_edge,
        super().__init__()
        self.node = node
        self.xyz_out = xyz_out
        self.rec_edge = input_edges_rec
        self.lig_edge = input_edges_lig
        self.len_rec = len_rec
        self.coord   =coord
        self.edge_index_rec=edge_index_rec
        self.edge_index_lig=edge_index_lig
        self.fold=fold
        self.setup()
        
    def setup(self, stage = None):
        counto=0
        together=[]
        self.data= []
        train_set=[]
        test_set=[]
        for i in range(len(self.node)):
            self.data.append([
                              self.node[i].float().clone().detach(),
                              self.xyz_out[i].float().clone().detach(),
                              self.coord[i].float().clone().detach(),
                              self.lig_edge[i].float().clone().detach(),
                              self.rec_edge[i].float().clone().detach(),
                              self.edge_index_lig[i].float().clone().detach(),
                              self.edge_index_rec[i].float().clone().detach(),
                              self.len_rec[i]])
        
        kf = KFold(n_splits=5, shuffle=True, random_state=11001)    
        for tog in kf.split(self.data):
            together=together+[tog]
        ti,vi=together[self.fold]
        self.tii, self.vii = ti.tolist(), vi.tolist()
        for i in self.vii:
            test_set.append(self.data[i])
        for i in self.tii:
            train_set.append(self.data[i])


        self.train_set,self.val_set=train_set,test_set

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=1,num_workers=32)

#     def val_dataloader(self):
#         return DataLoader(self.val_set, batch_size=1,num_workers=32)

    def test_dataloader(self):
        return DataLoader(self.val_set, batch_size=1,num_workers=32)



In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,4)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[3],max_epochs=10)
modelpres=LinkPred().load_from_checkpoint('kfoldboii_trial1.ckpt')
# trainer.fit(modelpres,dataobjl)
trainer.test(modelpres,dataobjl)

In [ ]:
#epipred  with rot

In [ ]:
fullsamples=np.load('/workspace/testEpi.npy')

xyzl=[]
xyz_outl = []
input_edges_rec = []
input_edges_lig = []
sample_filel = []
maked_inputl = []
len_rec = []
coord_all=[]
edge_index_rec=[]
edge_index_lig=[]
sample_file=[]
def sigm(g):
    return(1/(1+math.exp(-g/100)))
for i in range(len(fullsamples)):
    edgesrec = []
    edgeslig = []
#     if fullsamples[i]+'_' in samplesppi4:
    try:
        datain = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_input_feat.pt'))
        pssm = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_pssm.pt'))
        edgesrec = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_edge_rec.pt'))
        edgeslig = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_edge_lig.pt'))
        adjl = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_adj_rec.pt')) 
        adjr = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_adj_lig.pt')) 
        interface = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_interface.pt'))
    except:
        continue
    if not(datain.shape[0]==pssm.shape[0]==interface.shape[0] ) :# or  interface.sum()==0.:
        continue
    feat = torch.hstack((datain, pssm))
    rcc=feat[:, 46].cpu()
    unique, counts = np.unique(rcc, return_counts=True)
    coun = dict(zip(unique, counts))
    rec_c = coun[0.0]
    receptor = feat[:rec_c, 3:]
    ligand = feat[rec_c:, 3:]
    coord=feat[:,:3]
    if receptor.shape[0]>999 or receptor.shape[0]<75 or ligand.shape[0]>999 or ligand.shape[0]<75:
        continue
#     receptor = receptor.expand(receptor.shape[0], receptor.shape[0],receptor.shape[1]).permute(2,1,0)
#     ligand = ligand.expand(ligand.shape[0], ligand.shape[0],ligand.shape[1]).permute(2,1,0)
#     print(i,ligand.shape)
    len_rec.append(rec_c)
    input_edges_rec.append(adjr)
    input_edges_lig.append(adjl)
    edge_index_rec.append(adjr.nonzero().t().contiguous())
    edge_index_lig.append(adjl.nonzero().t().contiguous())
    for coor in coord:
        coor[0]=sigm(coor[0])
        coor[1]=sigm(coor[1])
        coor[2]=sigm(coor[2])

    x1,x2,x3 = torch.from_numpy(np.random.uniform(3.14/12, 3.14, 3))
    cos1=torch.cos(x1)
    cos2=torch.cos(x2)
    cos3=torch.cos(x3)
    sin1=torch.sin(x1)
    sin2=torch.sin(x2)
    sin3=torch.sin(x3)
    RMatrix=torch.tensor([[cos1*cos2*cos3-sin1*sin3,cos3*cos2*sin1+sin3+cos1,-cos3*sin2 ],\
                            [-sin3*cos2*cos1-cos3*sin1, -sin3*cos2*sin1+cos3*cos1, sin3*sin2],\
                            [sin2*cos1,sin2*sin1,cos2]])

    ligcoord=  ((RMatrix @ torch.tensor(coord[rec_c:,]).double().T)).T
    reccoord=coord[:rec_c,:]

    coord=torch.cat((ligcoord,reccoord),0)
    feat = feat[:,3:]
    xyzl.append(feat)
    coord_all.append(coord)
    sample_filel.append(i)#fullsamples.index(i))
#     input_edges_rec.append(edgesrec)
#     input_edges_lig.append(edgeslig)
    xyz_outl.append(torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_interface.pt')))
    sample_file.append((fullsamples[i]+'1',fullsamples[i]+'2'))

    #     break
    
len(xyzl), len(xyz_outl), len(len_rec)#, len(input_edges_lig)

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,4)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=10)
modelpres=LinkPred().load_from_checkpoint('kfoldboii_trial1.ckpt')
# trainer.fit(modelpres,dataobjl)
trainer.test(modelpres,dataobjl)

In [ ]:
#db5.5

In [ ]:
fullsamples=np.load('/workspace/db5samplelist.npy')

xyzl=[]
xyz_outl = []
input_edges_rec = []
input_edges_lig = []
sample_filel = []
maked_inputl = []
len_rec = []
coord_all=[]
edge_index_rec=[]
edge_index_lig=[]
def sigm(g):
    return(1/(1+math.exp(-g/100)))
for i in range(len(fullsamples)):
    edgesrec = []
    edgeslig = []
#     if fullsamples[i]+'_' in samplesppi4:
    try:
        try:
            datain= torch.load(os.path.join('/workspace/madhav/all_feat/ALL/'+str(fullsamples[i]).split('_')[0].upper()+'.'+str(fullsamples[i]).split('_')[1].split(':')[0]+'.'+str(fullsamples[i]).split('_')[1].split(':')[1]+'.'+'_input_feat.pt'))
        except:
            datain= torch.load(os.path.join('/workspace/madhav/all_feat/ALL/'+str(fullsamples[i]).split('_')[0].lower()+'.'+str(fullsamples[i]).split('_')[1].split(':')[0]+'.'+str(fullsamples[i]).split('_')[1].split(':')[1]+'.'+'_input_feat.pt')) 
        pssm = torch.load(os.path.join('/workspace/pssmdb5ppiprism',fullsamples[i]+'_pssm.pt'))
        edgesrec = torch.load(os.path.join('/workspace/InputEdge_file_nameedge_rec', fullsamples[i]+'_edge_rec.pt'))
        edgeslig = torch.load(os.path.join('/workspace/InputEdge_file_nameedge_lig', fullsamples[i]+'_edge_lig.pt'))
        adjl = torch.load(os.path.join('/workspace/inputfeatdb5ppiprism',fullsamples[i]+'_adj_rec.pt')) 
        adjr = torch.load(os.path.join('/workspace/inputfeatdb5ppiprism',fullsamples[i]+'_adj_lig.pt')) 
        interface = torch.load(os.path.join('/workspace/ReadyToPair', fullsamples[i]+'_interface.pt'))
    except:
        continue
    if not(datain.shape[0]==pssm.shape[0]==interface.shape[0] ) :# or  interface.sum()==0.:
        continue
    feat = torch.hstack((datain, pssm))
    rcc=feat[:, 46].cpu()
    unique, counts = np.unique(rcc, return_counts=True)
    coun = dict(zip(unique, counts))
    rec_c = coun[0.0]
    receptor = feat[:rec_c, 3:]
    ligand = feat[rec_c:, 3:]
    coord=feat[:,:3]
    if receptor.shape[0]>999 or receptor.shape[0]<75 or ligand.shape[0]>999 or ligand.shape[0]<75:
        continue
#     receptor = receptor.expand(receptor.shape[0], receptor.shape[0],receptor.shape[1]).permute(2,1,0)
#     ligand = ligand.expand(ligand.shape[0], ligand.shape[0],ligand.shape[1]).permute(2,1,0)
#     print(i,ligand.shape)
    len_rec.append(rec_c)
    input_edges_rec.append(adjr)
    input_edges_lig.append(adjl)
    edge_index_rec.append(adjr.nonzero().t().contiguous())
    edge_index_lig.append(adjl.nonzero().t().contiguous())
    for coor in coord:
        coor[0]=sigm(coor[0])
        coor[1]=sigm(coor[1])
        coor[2]=sigm(coor[2])


    feat = feat[:,3:]
    xyzl.append(feat)
    coord_all.append(coord)
    sample_filel.append(i)#fullsamples.index(i))
#     input_edges_rec.append(edgesrec)
#     input_edges_lig.append(edgeslig)
    xyz_outl.append(torch.load(os.path.join('/workspace/ReadyToPair', fullsamples[i]+'_interface.pt')))

    #     break
    
len(xyzl), len(xyz_outl), len(len_rec)#, len(input_edges_lig)

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,4)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=10)
modelpres=LinkPred().load_from_checkpoint('kfoldboii_trial1.ckpt')
# trainer.fit(modelpres,dataobjl)
trainer.test(modelpres,dataobjl)

In [ ]:
#TRAINIG ON EPIPRED FOR 3 EPOCHS 5 folds

In [ ]:
fullsamples=np.load('/workspace/testEpi.npy')

xyzl=[]
xyz_outl = []
input_edges_rec = []
input_edges_lig = []
sample_filel = []
maked_inputl = []
len_rec = []
coord_all=[]
edge_index_rec=[]
edge_index_lig=[]
sample_file=[]
def sigm(g):
    return(1/(1+math.exp(-g/100)))
for i in range(len(fullsamples)):
    edgesrec = []
    edgeslig = []
#     if fullsamples[i]+'_' in samplesppi4:
    try:
        datain = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_input_feat.pt'))
        pssm = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_pssm.pt'))
        edgesrec = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_edge_rec.pt'))
        edgeslig = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_edge_lig.pt'))
        adjl = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_adj_rec.pt')) 
        adjr = torch.load(os.path.join('/workspace/EpipredFeat',fullsamples[i]+'_adj_lig.pt')) 
        interface = torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_interface.pt'))
    except:
        continue
    if not(datain.shape[0]==pssm.shape[0]==interface.shape[0] ) :# or  interface.sum()==0.:
        continue
    feat = torch.hstack((datain, pssm))
    rcc=feat[:, 46].cpu()
    unique, counts = np.unique(rcc, return_counts=True)
    coun = dict(zip(unique, counts))
    rec_c = coun[0.0]
    receptor = feat[:rec_c, 3:]
    ligand = feat[rec_c:, 3:]
    coord=feat[:,:3]
    if receptor.shape[0]>999 or receptor.shape[0]<75 or ligand.shape[0]>999 or ligand.shape[0]<75:
        continue
#     receptor = receptor.expand(receptor.shape[0], receptor.shape[0],receptor.shape[1]).permute(2,1,0)
#     ligand = ligand.expand(ligand.shape[0], ligand.shape[0],ligand.shape[1]).permute(2,1,0)
#     print(i,ligand.shape)
    len_rec.append(rec_c)
    input_edges_rec.append(adjr)
    input_edges_lig.append(adjl)
    edge_index_rec.append(adjr.nonzero().t().contiguous())
    edge_index_lig.append(adjl.nonzero().t().contiguous())
    for coor in coord:
        coor[0]=sigm(coor[0])
        coor[1]=sigm(coor[1])
        coor[2]=sigm(coor[2])

    x1,x2,x3 = torch.from_numpy(np.random.uniform(3.14/12, 3.14, 3))
    cos1=torch.cos(x1)
    cos2=torch.cos(x2)
    cos3=torch.cos(x3)
    sin1=torch.sin(x1)
    sin2=torch.sin(x2)
    sin3=torch.sin(x3)
    RMatrix=torch.tensor([[cos1*cos2*cos3-sin1*sin3,cos3*cos2*sin1+sin3+cos1,-cos3*sin2 ],\
                            [-sin3*cos2*cos1-cos3*sin1, -sin3*cos2*sin1+cos3*cos1, sin3*sin2],\
                            [sin2*cos1,sin2*sin1,cos2]])

    ligcoord=  ((RMatrix @ torch.tensor(coord[rec_c:,]).double().T)).T
    reccoord=coord[:rec_c,:]

    coord=torch.cat((ligcoord,reccoord),0)
    feat = feat[:,3:]
    xyzl.append(feat)
    coord_all.append(coord)
    sample_filel.append(i)#fullsamples.index(i))
#     input_edges_rec.append(edgesrec)
#     input_edges_lig.append(edgeslig)
    xyz_outl.append(torch.load(os.path.join('/workspace/EpipredFeat', fullsamples[i]+'_interface.pt')))
    sample_file.append((fullsamples[i]+'1',fullsamples[i]+'2'))

    #     break
    
len(xyzl), len(xyz_outl), len(len_rec)#, len(input_edges_lig)

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,0)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=3)
modelpres=LinkPred().load_from_checkpoint('kfoldboii_trial1.ckpt')
trainer.fit(modelpres,dataobjl)
trainer.test(modelpres,dataobjl)
trainer.save_checkpoint("kfoldboii_trial1_epi.ckpt")

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,1)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=3)
modelpres=LinkPred().load_from_checkpoint('kfoldboii_trial1_epi.ckpt')
trainer.fit(modelpres,dataobjl)
trainer.test(modelpres,dataobjl)
trainer.save_checkpoint("kfoldboii_trial1_epi.ckpt")

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,2)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=3)
modelpres=LinkPred().load_from_checkpoint('kfoldboii_trial1_epi.ckpt')
trainer.fit(modelpres,dataobjl)
trainer.test(modelpres,dataobjl)
trainer.save_checkpoint("kfoldboii_trial1_epi.ckpt")

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,3)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=3)
modelpres=LinkPred().load_from_checkpoint('kfoldboii_trial1_epi.ckpt')
trainer.fit(modelpres,dataobjl)
trainer.test(modelpres,dataobjl)
trainer.save_checkpoint("kfoldboii_trial1_epi.ckpt")

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,4)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer=Trainer(gpus=[1],max_epochs=3)
modelpres=LinkPred().load_from_checkpoint('kfoldboii_trial1_epi.ckpt')
trainer.fit(modelpres,dataobjl)
trainer.test(modelpres,dataobjl)
trainer.save_checkpoint("kfoldboii_trial1_epi.ckpt")

In [ ]:
class ProteinDataL(pl.LightningDataModule):
    def __init__(self, node,  xyz_out, len_rec,input_edges_lig,input_edges_rec,coord,edge_index_rec,edge_index_lig,fold): #rec_edge, lig_edge,
        super().__init__()
        self.node = node
        self.xyz_out = xyz_out
        self.rec_edge = input_edges_rec
        self.lig_edge = input_edges_lig
        self.len_rec = len_rec
        self.coord   =coord
        self.edge_index_rec=edge_index_rec
        self.edge_index_lig=edge_index_lig
        self.fold=fold
        self.setup()
        
    def setup(self, stage = None):
        counto=0
        together=[]
        self.data= []
        train_set=[]
        test_set=[]
        for i in range(len(self.node)):
            self.data.append([
                              self.node[i].float().clone().detach(),
                              self.xyz_out[i].float().clone().detach(),
                              self.coord[i].float().clone().detach(),
                              self.lig_edge[i].float().clone().detach(),
                              self.rec_edge[i].float().clone().detach(),
                              self.edge_index_lig[i].float().clone().detach(),
                              self.edge_index_rec[i].float().clone().detach(),
                              self.len_rec[i]])
        
        kf = KFold(n_splits=5, shuffle=True, random_state=230696)    
        for tog in kf.split(self.data):
            together=together+[tog]
        ti,vi=together[self.fold]
        self.tii, self.vii = ti.tolist(), vi.tolist()
        for i in self.vii:
            test_set.append(self.data[i])
        for i in self.tii:
            train_set.append(self.data[i])


        self.train_set,self.val_set=train_set,test_set

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=1,num_workers=32)

#     def val_dataloader(self):
#         return DataLoader(self.val_set, batch_size=1,num_workers=32)

    def test_dataloader(self):
        return DataLoader(self.val_set, batch_size=1,num_workers=32)

In [ ]:
dataobjl = ProteinDataL( xyzl,  xyz_outl, len_rec,input_edges_lig,input_edges_rec,coord_all,edge_index_rec,edge_index_lig,4)#.to(device)  #edge_list, input_edges_rec, input_edges_lig,
dataobjl.setup()
trainer.test(modelpres,dataobjl)
